In [1]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2025-03-13 16:23:26--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M   111MB/s    in 0.2s    

2025-03-13 16:23:26 (111 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [3]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [13]:
# add your code here - consider creating a new cell for each section of code
user_counts = df_ratings['user'].value_counts()
df_user = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

book_counts = df_ratings['isbn'].value_counts()
df_book = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]

df_book = pd.merge(df_book, df_books[['isbn', 'title']], on='isbn')

df = pd.merge(df_user, df_book[['isbn', 'title']], on='isbn')

df

,user,isbn,rating,title
0,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...
1,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...
2,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...
3,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...
4,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...
...,...,...,...,...
12472504,275970,1400031362,0.0,Morality for Beautiful Girls (No.1 Ladies Dete...
12472505,275970,1400031362,0.0,Morality for Beautiful Girls (No.1 Ladies Dete...
12472506,275970,1400031362,0.0,Morality for Beautiful Girls (No.1 Ladies Dete...
12472507,275970,1400031362,0.0,Morality for Beautiful Girls (No.1 Ladies Dete...


In [16]:
df_matrix = df.pivot_table(index='title', columns='user', values='rating').fillna(0)

df_sparse = csr_matrix(df_matrix.values)

df_index = pd.Series(df_matrix.index)

In [20]:
model_knn = NearestNeighbors(metric='cosine', algorithm='brute', n_neighbors=6)

model_knn.fit(df_sparse)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=6)

In [40]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  recommended_books = []
  book_index = df_index[df_index == book].index[0]
  distances, indexes = model_knn.kneighbors(df_sparse[book_index], n_neighbors=6)
  for i in range(1,6):
    recommended_book_index = indexes[0][-i]
    recommended_book_title = df_index[recommended_book_index]
    recommended_book_distance = distances[0][-i]
    recommended_books.append([recommended_book_title, recommended_book_distance])

  return [book, recommended_books]

In [41]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
